In [138]:
import numpy as np
import random
import math

In [139]:
def generate_vrptw_data(
    num_customers,
    max_demand,
    max_travel_time,
    max_service_time,
    max_time_window,
    max_cost,
    random_seed=None
):
    if random_seed is not None:
        random.seed(random_seed)
        np.random.seed(random_seed)

    demands = np.array(
        [0] + [random.randint(1, max_demand) for _ in range(num_customers)]
    )

    travel_times = np.zeros((num_customers + 1, num_customers + 1), dtype=int)
    for i in range(num_customers + 1):
        for j in range(i + 1, num_customers + 1):
            travel_times[i, j] = travel_times[j, i] = random.randint(1, max_travel_time)

    costs = np.zeros((num_customers + 1, num_customers + 1), dtype=int)
    for i in range(num_customers + 1):
        for j in range(i + 1, num_customers + 1):
            costs[i, j] = costs[j, i] = random.randint(1, max_cost)

    time_windows = np.array(
        [(0, max_time_window)]
        + [
            (
                random.randint(0, max_time_window // 2),
                random.randint(max_time_window // 2, max_time_window),
            )
            for _ in range(num_customers)
        ]
    )

    service_times = np.array(
        [0] + [random.randint(1, max_service_time) for _ in range(num_customers)]
    )

    return demands, travel_times, costs, time_windows, service_times

In [140]:
max_demand = 5
max_travel_time = 30
max_service_time = 15
max_time_window = 100
N = 10  # Nombre de points de collecte (sans compter le dépôt)
num_camions = 2
camion_capacity = 15
max_cost = 50

demands, travel_times, costs, time_windows, service_times = generate_vrptw_data(
    N,
    max_demand,
    max_travel_time,
    max_service_time,
    max_time_window,
    max_cost,
    random_seed=42,
)
print("Demands:", demands)
print("Travel Times:\n", travel_times)
print("Costs:\n", costs)
print("Time Windows:", time_windows)
print("Service Times:", service_times)

Demands: [0 1 1 3 2 2 2 1 5 1 5]
Travel Times:
 [[ 0 14  2  1  3  7  8 17 20  1 18]
 [14  0  7 23 21 23 18 14  8 15 19]
 [ 2  7  0  9 26 28  1 25 26  6 23]
 [ 1 23  9  0 14 11  9  5  7 25 11]
 [ 3 21 26 14  0  4  3 13  4 12 28]
 [ 7 23 28 11  4  0 12 20  9 26  2]
 [ 8 18  1  9  3 12  0 24 15 18  4]
 [17 14 25  5 13 20 24  0 30 13  3]
 [20  8 26  7  4  9 15 30  0 18 10]
 [ 1 15  6 25 12 26 18 13 18  0 27]
 [18 19 23 11 28  2  4  3 10 27  0]]
Costs:
 [[ 0 41 40 24 37 13 46  5  3 43 15]
 [41  0 50 19  6 15  7 25 18 30 41]
 [40 50  0 24 11 24 23 14 43 18 45]
 [24 19 24  0 44 42  5 39 41 11 35]
 [37  6 11 44  0 47 16 11 30 25 18]
 [13 15 24 42 47  0 41 45 36 15 44]
 [46  7 23  5 16 41  0 21 50 50  4]
 [ 5 25 14 39 11 45 21  0 15  3 21]
 [ 3 18 43 41 30 36 50 15  0 26 18]
 [43 30 18 11 25 15 50  3 26  0  5]
 [15 41 45 35 18 44  4 21 18  5  0]]
Time Windows: [[  0 100]
 [ 13  86]
 [ 45  70]
 [ 13  91]
 [ 31  75]
 [ 41  79]
 [  9  66]
 [  8  65]
 [ 47  85]
 [ 34  66]
 [ 47  87]]
Service Times:

## Simulated Annealing

Constants for Simulated Annealing

In [141]:
initial_temperature = 100000
cooling_rate = 0.995
num_iterations = 1000

Define the objective function for VRPTW (using costs, not travel times)

In [142]:
def calculate_total_cost(solution, costs):
    total_cost = 0
    for route in solution:
        for i in range(len(route) - 1):
            total_cost += costs[route[i]][route[i + 1]]
        total_cost += costs[route[-1]][0]  # Return to depot cost
    return total_cost

Calculate the cost for the previews cost as a sanity test

In [143]:
calculate_total_cost(solution=[[0, 7, 0], [0, 8, 0]], costs=costs)

np.int64(16)

Generate a random feasible solution

In [144]:
def generate_initial_solution(num_customers, num_vehicles, demands, camion_capacity):
    customers = list(range(1, num_customers + 1))
    random.shuffle(customers)

    solution = []
    vehicle_route = []
    vehicle_load = 0
    for customer in customers:
        if vehicle_load + demands[customer] <= camion_capacity:
            vehicle_route.append(customer)
            vehicle_load += demands[customer]
        else:
            solution.append(vehicle_route)
            vehicle_route = [customer]
            vehicle_load = demands[customer]
    solution.append(vehicle_route)
    #solution = [[0, 7, 0], [0, 8, 0]]
    return solution

Check if a solution respects the time windows and capacity constraints

In [145]:
def is_feasible(
    solution, demands, camion_capacity, time_windows, travel_times, service_times
):
    for route in solution:
        load = 0
        time = 0
        for i in range(len(route)):
            customer = route[i]
            load += demands[customer]
            if load > camion_capacity:
                return False
            if i > 0:
                time += (
                    travel_times[route[i - 1]][customer] + service_times[route[i - 1]]
                )
            if time < time_windows[customer][0] or time > time_windows[customer][1]:
                return False
        time += travel_times[route[-1]][0]  # Return to depot
    return True

Apply a small modification (swap two customers between routes) to generate a neighbor solution

In [146]:
def get_neighbor(solution):
    new_solution = [route[:] for route in solution]  # Deep copy
    route1, route2 = random.sample(new_solution, 2)
    if route1 and route2:
        i, j = random.randint(0, len(route1) - 1), random.randint(0, len(route2) - 1)
        route1[i], route2[j] = route2[j], route1[i]
    return new_solution

Simulated Annealing

In [147]:
def simulated_annealing(
    num_customers,
    num_vehicles,
    demands,
    camion_capacity,
    costs,
    travel_times,
    time_windows,
    service_times,
):
    current_solution = generate_initial_solution(
        num_customers, num_vehicles, demands, camion_capacity
    )
    print(f'Initial_solution: {current_solution}')
    best_solution = current_solution
    best_cost = calculate_total_cost(current_solution, costs)
    temperature = initial_temperature

    for _ in range(num_iterations):
        new_solution = get_neighbor(current_solution)
        if is_feasible(
            new_solution,
            demands,
            camion_capacity,
            time_windows,
            travel_times,
            service_times,
        ):
            new_cost = calculate_total_cost(new_solution, costs)
            cost_diff = new_cost - best_cost
            if cost_diff < 0 or random.random() < np.exp(-cost_diff / temperature):
                current_solution = new_solution
                if new_cost < best_cost:
                    best_cost = new_cost
                    best_solution = new_solution
        temperature *= cooling_rate  # Cool down

    return best_solution, best_cost

In [148]:
# Example usage
solution, cost = simulated_annealing(
    N,
    num_camions,
    demands,
    camion_capacity,
    costs,  # Using costs matrix
    travel_times,  # Still used in time window constraints
    time_windows,
    service_times,
)

print("Best solution:", solution)
print("Best cost:", cost)

Initial_solution: [[5, 8, 7, 10, 4], [9, 6, 3, 2, 1]]
Best solution: [[5, 8, 7, 10, 4], [9, 6, 3, 2, 1]]
Best cost: 297
